# Descripción de la Práctica a realizar


El objetivo de esta actividad será la creación de un dataset a partir de los datos
contenidos en una web.

In [273]:
import requests
from urllib.error import HTTPError
from urllib.error import URLError
#Beautiful Soup is a Python library for pulling data out of HTML and XML files. 
#It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree.
#import sys
#!{sys.executable} -m pip install bs4
#import gensim
from bs4 import BeautifulSoup

#Url de la web empleada para obtener precios de la gasolina para los municipios
url = "https://gasolinabarata.info"


In [274]:
#Método que obtiene el listado de ciudades españolas con sus links
def getCities(url,numDivToProcess,textTitle,nameCity):
    
    dictionaryCities={}
    
    print("Buscamos en la página:"+url+", los enlaces de las provincias de España")
    page = requests.get(url)

    res = BeautifulSoup(page.content,"html.parser")

    print('En dicha URL, Buscamos los divs que contengan la clase div.listGasolineras')
    divCities=res.find_all("div", class_="listGasolineras")
    
    print("Hay "+str(len(divCities))+ " divs, hay que coger el "+str(numDivToProcess)+", el de provincias")
    
    if len(divCities)==0 or "Enlaces internos" in divCities[numDivToProcess-1].text:
        print("Enlace Interno a propia ciudad")
        print(url)
        print(nameCity)
        dictionaryCities[nameCity]=url
    else:
        a_cities=divCities[numDivToProcess-1].find_all("a")
        print("Una vez obtenido el div donde se encuentran los enlaces de las provincias, tenemos que leer cada enlace:")

        for a in a_cities:
            if a.has_attr('href'):
                link=a.attrs['href']
                city=a.attrs['title'].replace(textTitle,"")
                print(link)
                print(city)
                dictionaryCities[city]=link
    return dictionaryCities

In [275]:
import datetime
#Obtener precios del combustible
def getGasoil(url):
    #Asignamos fecha de obtención del precio
    dictionaryPrices={'Fecha': datetime.datetime.now()}
    try:
        print("Buscamos en la página:"+url+", los precios de combustible de los municipios.")
        page = requests.get(url)
    except HTTPError as e:
        print(e)
    except URLError:
        print("Servidor caido dominio incorrecto")
    else:
        #Obtenemos el contenido de la página
        res = BeautifulSoup(page.content,"html.parser")
        #Si la página retornada es un 404, no pudo encontrar en dicha página el div que contiene precios
        if "The resource requested could not be found on this server!" in res.text:
            print('No se pudo encontrar DIV en '+url)
        else:
            print('En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina')
            
            #Obtenemos div contenedor de precios
            divPrices=res.find("div", class_="bloqueGasolina precioGasolina")
            
            #Si el div no contiene nada, porque la página no tiene datos, no puede encontrar el div de precios
            if divPrices==None:
                print('No se pudo encontrar DIV en '+url)
            else:
                #Si contiene precios el div, tenemos que trocear los div precios
                print("Una vez obtenido el div donde se encuentran los precios leemos cada precio:")
                
                for div in divPrices:
                    #Cada div compuesto por dos div contiene un div para el titulo y otro para el precio
                    if(len(div)==2):
                        price=div.find_all('div')
                        #Partimos cada dupla de div en clave / valor
                        dictionaryPrices[price[0].text.replace("Precio ","")]=price[1].text.replace("Precio ","")
                        print(div.find_all('div'))
    return dictionaryPrices

In [276]:
#Convertimos el precio obtenido en texto, tratando aquellos precios no encontrados, como elementos vacios
def getPriceObject(price,key):
    result=None
    if key in price:
        result=price[key]
    return result

In [281]:
import time

print('Inicio:'+str(datetime.datetime.now()))
cities_towns={}
prices_town={}
data=[]

print('Obtenemos las provincias de España')
cities=getCities(url,2,"Gasolineras en ",None)

print('Por cada provincia, buscaremos sus municipios')
for city in cities:
    #Esperamos 5 segundos por cada ciudad
    time.sleep(5)
    print()
    print(city)
    print(str(datetime.datetime.now()))
    
    print('Obtenemos los municipios de '+city)
    #Obtenemos los enlaces de cada uno de los municipios de cada ciudad
    towns=getCities(cities[city],1,"Gasolineras ",city)
    #A cada ciudad le asignamos su lista de municipios (nombre municipio, enlace)
    cities_towns[city]=towns
    for town in towns:
        print()
        print(town)
        print()
        #Obtenemos precios del municipio
        prices=getGasoil(cities_towns[city][town])
        
        prices['City']=city
        prices_town[town]=prices
        
        #Agregamos al modelo data, la información
        data.append({'City':city
                      ,'Town':town
                      ,'gasolina 95':getPriceObject(prices,'gasolina 95')
                      ,'gasolina 98':getPriceObject(prices,'gasolina 98')
                      ,'diésel':getPriceObject(prices,'diésel')
                      ,'diésel plus':getPriceObject(prices,'diésel plus')
                      ,'gasóleo B':getPriceObject(prices,'gasóleo B')
                      ,'biodiésel':getPriceObject(prices,'biodiésel')
                      ,'GNL':getPriceObject(prices,'GNL')
                      ,'GNC':getPriceObject(prices,'GNC')
                      ,'GLP':getPriceObject(prices,'GLP')
                      ,'Fecha':datetime.datetime.now()
                     })
print('Fin:'+str(datetime.datetime.now()))

Inicio:2020-04-05 18:07:34.084690
Obtenemos las provincias de España
Buscamos en la página:https://gasolinabarata.info, los enlaces de las provincias de España
En dicha URL, Buscamos los divs que contengan la clase div.listGasolineras
Hay 2 divs, hay que coger el 2, el de provincias
Una vez obtenido el div donde se encuentran los enlaces de las provincias, tenemos que leer cada enlace:
https://gasolinabarata.info/pais-vasco/alava/
Álava
https://gasolinabarata.info/castilla-la-mancha/albacete/
Albacete
https://gasolinabarata.info/valencia/alicante/
Alicante
https://gasolinabarata.info/andalucia/almeria/
Almería
https://gasolinabarata.info/asturias/
Asturias
https://gasolinabarata.info/castilla-leon/avila/
Ávila
https://gasolinabarata.info/extremadura/badajoz/
Badajoz
https://gasolinabarata.info/cataluna/barcelona/
Barcelona
https://gasolinabarata.info/castilla-leon/burgos/
Burgos
https://gasolinabarata.info/extremadura/caceres/
Cáceres
https://gasolinabarata.info/andalucia/cadiz/
Cádiz


En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,109 €/l</div>]
[<div>Precio diésel</div>, <div>1,038 €/l</div>]
[<div>Precio diésel plus</div>, <div>1,109 €/l</div>]

Campezo/Kanpezu

Buscamos en la página:https://gasolinabarata.info/pais-vasco/alava/campezo-kanpezu/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,100 €/l</div>]
[<div>Precio diésel</div>, <div>1,000 €/l</div>]
[<div>Precio diésel plus</div>, <div>1,200 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,760 €/l</div>]

Elburgo/Burgelu

Buscamos en la página:https://gasolinabarata.info/pais-vasco/alava/elburgo-burgelu/, los precios de combustible de los municipios.
En dicha U

En dicha URL, Buscamos los divs que contengan la clase div.listGasolineras
Hay 2 divs, hay que coger el 1, el de provincias
Una vez obtenido el div donde se encuentran los enlaces de las provincias, tenemos que leer cada enlace:
https://gasolinabarata.info/castilla-la-mancha/albacete/abengibre/
Abengibre
https://gasolinabarata.info/castilla-la-mancha/albacete/alatoz/
Alatoz
https://gasolinabarata.info/castilla-la-mancha/albacete/albacete/
Albacete
https://gasolinabarata.info/castilla-la-mancha/albacete/alcadozo/
Alcadozo
https://gasolinabarata.info/castilla-la-mancha/albacete/alcala-del-jucar/
Alcalá del Júcar
https://gasolinabarata.info/castilla-la-mancha/albacete/alcaraz/
Alcaraz
https://gasolinabarata.info/castilla-la-mancha/albacete/almansa/
Almansa
https://gasolinabarata.info/castilla-la-mancha/albacete/alpera/
Alpera
https://gasolinabarata.info/castilla-la-mancha/albacete/balazote/
Balazote
https://gasolinabarata.info/castilla-la-mancha/albacete/barrax/
Barrax
https://gasolinabar

En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,159 €/l</div>]
[<div>Precio diésel</div>, <div>1,099 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,810 €/l</div>]

Barrax

Buscamos en la página:https://gasolinabarata.info/castilla-la-mancha/albacete/barrax/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,159 €/l</div>]
[<div>Precio diésel</div>, <div>1,048 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,684 €/l</div>]

Bienservida

Buscamos en la página:https://gasolinabarata.info/castilla-la-mancha/albacete/bienservida/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolin

En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,219 €/l</div>]
[<div>Precio diésel</div>, <div>1,119 €/l</div>]

Madrigueras

Buscamos en la página:https://gasolinabarata.info/castilla-la-mancha/albacete/madrigueras/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,067 €/l</div>]
[<div>Precio gasolina 98</div>, <div>1,209 €/l</div>]
[<div>Precio diésel</div>, <div>1,072 €/l</div>]
[<div>Precio diésel plus</div>, <div>1,119 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,735 €/l</div>]

Minaya

Buscamos en la página:https://gasolinabarata.info/castilla-la-mancha/albacete/minaya/, los precios de combustible de los municipios.
En dicha URL, 

...

En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,159 €/l</div>]
[<div>Precio gasolina 98</div>, <div>1,259 €/l</div>]
[<div>Precio diésel</div>, <div>1,099 €/l</div>]
[<div>Precio diésel plus</div>, <div>1,159 €/l</div>]

Aguarón

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/aguaron/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,260 €/l</div>]
[<div>Precio diésel</div>, <div>1,200 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,760 €/l</div>]

Ainzón

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/ainzon/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la

En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>0,972 €/l</div>]
[<div>Precio diésel</div>, <div>0,897 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,589 €/l</div>]

Bulbuente

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/bulbuente/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,319 €/l</div>]
[<div>Precio diésel</div>, <div>1,199 €/l</div>]
[<div>Precio diésel plus</div>, <div>1,309 €/l</div>]

El Burgo de Ebro

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/burgo-de-ebro-el/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.prec

En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,299 €/l</div>]
[<div>Precio diésel</div>, <div>1,189 €/l</div>]

El Frasno

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/frasno-el/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,289 €/l</div>]
[<div>Precio gasolina 98</div>, <div>1,379 €/l</div>]
[<div>Precio diésel</div>, <div>1,209 €/l</div>]

Fuendejalón

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/fuendejalon/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los pre

En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,125 €/l</div>]
[<div>Precio gasolina 98</div>, <div>1,273 €/l</div>]
[<div>Precio diésel</div>, <div>1,027 €/l</div>]
[<div>Precio diésel plus</div>, <div>1,129 €/l</div>]
[<div>Precio GLP</div>, <div>0,679 €/l</div>]

Munébrega

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/munebrega/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,180 €/l</div>]
[<div>Precio gasolina 98</div>, <div>1,260 €/l</div>]
[<div>Precio diésel</div>, <div>1,070 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,640 €/l</div>]

Nonaspe

Buscamos en la página:https://gasolinabarata.info/aragon/zarag

En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,067 €/l</div>]
[<div>Precio diésel</div>, <div>0,946 €/l</div>]
[<div>Precio gasóleo B</div>, <div>0,572 €/l</div>]

Sobradiel

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/sobradiel/, los precios de combustible de los municipios.
En dicha URL, Buscamos el div que contengan la clase div.bloqueGasolina.precioGasolina
Una vez obtenido el div donde se encuentran los precios leemos cada precio:
[<div>Precio gasolina 95</div>, <div>1,149 €/l</div>]
[<div>Precio gasolina 98</div>, <div>1,249 €/l</div>]
[<div>Precio diésel</div>, <div>1,079 €/l</div>]
[<div>Precio diésel plus</div>, <div>1,139 €/l</div>]

Sos del Rey Católico

Buscamos en la página:https://gasolinabarata.info/aragon/zaragoza/sos-del-rey-catolico/, los precios de combustible de los municipios.
En dicha URL, 

In [282]:
import pandas as pd

print('Creamos Dataframe con los resultados')
Precio_Medio_Combustible_Municipios_ES = pd.DataFrame(columns=['City', 'Town','gasolina 95','gasolina 98','diésel','diésel plus','gasóleo B','biodiésel','GNL','GNC','GLP','Fecha']
                  ,data=data)
Precio_Medio_Combustible_Municipios_ES

Creamos Dataframe con los resultados


,City,Town,gasolina 95,gasolina 98,diésel,diésel plus,gasóleo B,biodiésel,GNL,GNC,GLP,Fecha
0,Álava,Alegría-Dulantzi,"1,199 €/l",None,"1,109 €/l",None,"0,726 €/l",None,None,None,None,2020-04-05 18:07:41.365654
1,Álava,Amurrio,"1,117 €/l","1,249 €/l","1,044 €/l","1,099 €/l",None,None,None,None,None,2020-04-05 18:07:41.789597
2,Álava,Arraia-Maeztu,"1,250 €/l","1,315 €/l","1,145 €/l",None,None,None,None,None,None,2020-04-05 18:07:42.176546
3,Álava,Arrazua-Ubarrundia,"1,152 €/l","1,276 €/l","1,084 €/l","1,154 €/l",None,"1,159 €/l",None,None,None,2020-04-05 18:07:43.312640
4,Álava,Artziniega,"1,165 €/l","1,285 €/l","1,085 €/l","1,125 €/l","0,660 €/l",None,None,None,None,2020-04-05 18:07:43.648186
...,...,...,...,...,...,...,...,...,...,...,...,...
3065,Zaragoza,Villanueva de Gállego,"1,111 €/l","1,247 €/l","1,031 €/l","1,147 €/l","0,809 €/l",None,None,None,None,2020-04-05 18:40:52.931560
3066,Zaragoza,Villanueva de Huerva,"1,169 €/l",None,"1,059 €/l","1,249 €/l","0,679 €/l",None,None,None,None,2020-04-05 18:40:53.341670
3067,Zaragoza,Villarroya de la Sierra,"1,079 €/l","1,179 €/l","1,025 €/l","1,095 €/l","0,763 €/l",None,None,None,None,2020-04-05 18:40:53.878471
3068,Zaragoza,Zaragoza,"1,107 €/l","1,246 €/l","1,030 €/l","1,118 €/l","0,727 €/l",None,"0,869 €/l","0,890 €/l","0,685 €/l",2020-04-05 18:40:54.362681


In [283]:
Precio_Medio_Combustible_Municipios_ES.info

<bound method DataFrame.info of           City                     Town gasolina 95 gasolina 98     diésel  \
0        Álava         Alegría-Dulantzi   1,199 €/l        None  1,109 €/l   
1        Álava                  Amurrio   1,117 €/l   1,249 €/l  1,044 €/l   
2        Álava            Arraia-Maeztu   1,250 €/l   1,315 €/l  1,145 €/l   
3        Álava       Arrazua-Ubarrundia   1,152 €/l   1,276 €/l  1,084 €/l   
4        Álava               Artziniega   1,165 €/l   1,285 €/l  1,085 €/l   
...        ...                      ...         ...         ...        ...   
3065  Zaragoza    Villanueva de Gállego   1,111 €/l   1,247 €/l  1,031 €/l   
3066  Zaragoza     Villanueva de Huerva   1,169 €/l        None  1,059 €/l   
3067  Zaragoza  Villarroya de la Sierra   1,079 €/l   1,179 €/l  1,025 €/l   
3068  Zaragoza                 Zaragoza   1,107 €/l   1,246 €/l  1,030 €/l   
3069  Zaragoza                    Zuera   1,051 €/l   1,239 €/l  0,979 €/l   

     diésel plus  gasóleo B  bi

In [284]:
Precio_Medio_Combustible_Municipios_ES.to_csv('Precio_Medio_Combustible_Municipios_ES.csv',sep='\t')